In [1]:

'''
---------------------------------------------------------------
Simulation for Function-on-Function Linear Regression
---------------------------------------------------------------
* 1 d

* 1 Response Y & 3 Predictor (X1, X2, X3)   

* Regular Dense Grid (num of grid: 21)

* Training samp size: 200
  Test     samp size: 100

* Using lpr (cpu version)

* Using Gaussian Kernal Function in LPR

* Using (real) Leave-One-Curve-Out CV for bandwidth selection in LPR 
---------------------------------------------------------------
'''

#####
import GetData
import FFReg
import Plotting
import lpr

###
import numpy as np
import time
from collections import namedtuple

# ---------
# Settings
# ---------
sim_rep_time = 100

tran_samp_size = 200
test_samp_size = 100
samp_size = tran_samp_size + test_samp_size

range_num_pts = np.array([4, 6])
num_pts = range_num_pts[1]

# -------------
# Setting of X1
# -------------
def X1_Mean_Func(s):
    #input, output: vector
    return(s /  + 2 * np.sin(s))


def Eigen_Funcs_X1(s):
    #input  :d-vector
    #output:array with dim (2 * d)
    return(np.array([np.sqrt(2 / 5) * -np.cos(2 * np.pi * s / 5), 
                     np.sqrt(2 / 5) *  np.sin(2 * np.pi * s / 5)]))

num_grid_X1 = 21
range_X1 = np.array([0, 5])
X1_eigen_val = np.array([9, 4])


# In[4]:

# -------------
# Setting of X2
# -------------
def X2_Mean_Func(s):
    #input, output: vector
    return(s / 5 + np.cos(1.5 * s))

def Eigen_Funcs_X2(s):
    #input  :d-vector
    #output:array with dim (2 * d) 
    return(np.array([-np.sqrt(2 / 10) * np.cos(4 * np.pi * s / 10), 
                     np.sqrt(2 / 10) *  np.sin(4 * np.pi * s / 10)]))

num_grid_X2 = 21

range_X2 = np.array([0, 10])

X2_eigen_val = np.array([12, 8])

# -------------
# Setting of X3
# -------------
def X3_Mean_Func(s):
    #input, output: vector
    return(np.sqrt(s) + np.cos(s))

def Eigen_Funcs_X3(s):
    #input  :d-vector
    #output:array with dim (2 * d) 
    return(np.array([-np.sqrt(2 / 10) * np.cos(np.pi * s / 10), 
                      np.sqrt(2 / 10) * np.sin(np.pi * s / 10)]))

num_grid_X3 = 21

range_X3 = np.array([0, 10])

X3_eigen_val = np.array([6, 4])

# -------------
# Setting of Y
# -------------

def Y_Mean_Func(s):
    #input, output: vector
    return(s / 2 + np.sin(s))

def Eigen_Funcs_Y(s):
    #input  :d-vector
    #output:array with dim (2 * d)
    return(np.array([np.sqrt(2 / 5) * -np.cos(np.pi * s / 5), 
                     np.sqrt(2 / 5) * np.sin(np.pi * s / 5)])) 

num_grid_Y = 21

range_Y = np.array([0, 5])

B_1 = np.array([[1, 0.8], [-1, 0.5]])
B_2 = np.array([[1, 1.5], [1.2, 0.5]])
B_3 = np.array([[0.6, 1.5], [-1.2, 1]])

In [2]:
time_grid_X1 = np.linspace(0, 5, 21)
time_grid_X2 = np.linspace(0, 10, 21)
time_grid_X3 = np.linspace(0, 10, 21)
time_grid_Y = np.linspace(0, 5, 21)


In [3]:
# bw for X1
X1_candidate_h_mean = np.arange(0.3, 1, 0.05).reshape(-1, 1)
X1_candidate_h_cov = np.arange(0.3, 0.5, 0.05).repeat(2).reshape(-1, 2)
X1_candidate_h_diag_cov = np.arange(0.3, 0.8, 0.05).reshape(-1, 1)

X2_candidate_h_mean = np.arange(0.3, 2, 0.05).reshape(-1, 1)
X2_candidate_h_cov = np.arange(0.3, 1.5, 0.1).repeat(2).reshape(-1, 2)
X2_candidate_h_diag_cov =  np.arange(0.3, 1.6, 0.1).reshape(-1, 1)


X3_candidate_h_mean = np.arange(0.3, 1, 0.05).reshape(-1, 1)
X3_candidate_h_cov = np.arange(0.3, 1, 0.1).repeat(2).reshape(-1, 2)
X3_candidate_h_diag_cov =  np.arange(0.3, 1, 0.1).reshape(-1, 1)

Y_candidate_h_mean = np.arange(0.3, 0.6, 0.05).reshape(-1, 1)
Y_candidate_h_cov = np.arange(0.3, 0.6, 0.05).repeat(2).reshape(-1, 2)

# bandwidth
X1_Y_candidate_h_cov = np.asanyarray(np.meshgrid(np.linspace(0.2, 0.4, 3), 
                                                 np.linspace(0.2, 0.4, 3))).T.reshape(-1,2)

# bandwidth
X2_Y_candidate_h_cov = np.asanyarray(np.meshgrid(np.linspace(0.4, 0.8, 5), 
                                                 np.linspace(0.1, 0.3, 3))).T.reshape(-1,2)

X3_Y_candidate_h_cov = np.asanyarray(np.meshgrid(np.linspace(0.3, 0.7, 3), 
                                                np.linspace(0.2, 0.4, 3))).T.reshape(-1,2)

X1_X2_candidate_h_cov = np.asanyarray(np.meshgrid(np.linspace(0.1, 0.6, 6), 
                                                np.linspace(0.3, 0.8, 6))).T.reshape(-1,2)


X1_X3_candidate_h_cov = np.asanyarray(np.meshgrid(np.linspace(0.2, 0.7, 6), 
                                                np.linspace(0.3, 1, 8))).T.reshape(-1,2)

X2_X3_candidate_h_cov = np.asanyarray(np.meshgrid(np.linspace(0.2, 0.7, 6), 
                                                np.linspace(0.3, 0.7, 5))).T.reshape(-1,2)


In [4]:
sim_rep_time = 100

In [5]:
name = ['X1', 'X2', 'X3', 'Y']

In [ ]:
my_MISE = np.zeros(sim_rep_time)
np.random.seed(104)
X1_data = []
X2_data = []
X3_data = []
Y_data = []

for i in range(sim_rep_time):
    start_time = time.time()
#     X1_data.append(GetData.Get_X_data(Mean_Func  = X1_Mean_Func, 
#                                  Eigen_Funcs = Eigen_Funcs_X1, 
#                                  eigen_val  = X1_eigen_val, 
#                                  domain = range_X1, 
#                                  tran_size  = tran_samp_size, 
#                                  test_size  = test_samp_size,
#                                  num_grid = num_grid_X1,
#                                  num_pts = range_num_pts,
#                                  sparse = True))


#     # In[8]:

#     X2_data.append(GetData.Get_X_data(Mean_Func  = X2_Mean_Func, 
#                                  Eigen_Funcs = Eigen_Funcs_X2, 
#                                  eigen_val  = X2_eigen_val, 
#                                  domain = range_X2, 
#                                  tran_size  = tran_samp_size, 
#                                  test_size  = test_samp_size,
#                                  num_grid = num_grid_X2,
#                                  num_pts = range_num_pts,
#                                  sparse = True))


#     # In[9]:

#     X3_data.append(GetData.Get_X_data(Mean_Func  = X3_Mean_Func, 
#                                  Eigen_Funcs = Eigen_Funcs_X3, 
#                                  eigen_val  = X3_eigen_val, 
#                                  domain = range_X3, 
#                                  tran_size  = tran_samp_size, 
#                                  test_size  = test_samp_size,
#                                  num_grid = num_grid_X3, 
#                                  num_pts = range_num_pts,
#                                  sparse = True))


#     # In[10]:

#     #Generate Simulation data of Y

#     Y_data.append(GetData.Get_Y_data(Mean_Func = Y_Mean_Func, 
#                                 Eigen_Funcs = Eigen_Funcs_Y,
#                                 X_fpcs = (X1_data[i].fpcs, X2_data[i].fpcs, X3_data[i].fpcs),
#                                 B = (B_1, B_2, B_3), 
#                                 domain = range_Y, 
#                                 test_size = test_samp_size, 
#                                 num_grid = num_grid_Y, 
#                                 num_pts = range_num_pts,
#                                  sparse = True))
                  
#     datas = [X1_data[i], X2_data[i], X3_data[i], Y_data[i]]
#     for j in range(len(datas)):
#         np.savetxt('tran '+ name[j] + ' of samp ' + np.str(i + 1), np.column_stack((datas[j].obs_tran)), 
#                    newline='\n' ,delimiter = ' ', fmt="%s")
        
#         np.savetxt('test '+ name[j] + ' of samp ' + np.str(i + 1), np.column_stack((datas[j].obs_test)), 
#                    newline='\n', delimiter = ' ', fmt="%s")
        
#         np.savetxt('real_on_grid '+ name[j] + ' of samp ' + np.str(i + 1), np.column_stack((datas[j].real_val_on_grid)), 
#                    newline='\n', delimiter = ' ', fmt="%s")
        
#         np.savetxt('tran_time_pt '+ name[j] + ' of samp ' + np.str(i + 1), np.column_stack((datas[j].tran_time_pts)), 
#                    newline='\n', delimiter = ' ', fmt="%s")
        
#         np.savetxt('test_time_pt '+ name[j] + ' of samp ' + np.str(i + 1), np.column_stack((datas[j].test_time_pts)), 
#                    newline='\n', delimiter = ' ', fmt="%s")
    
    #load_data
    datas = []
    datas.append(namedtuple('X1_data', ['obs_tran', 'obs_test', 'real_val_on_grid', 'tran_time_pts', 'test_time_pts', 'time_grid'], 
                             verbose=False))
    datas.append(namedtuple('X2_data', ['obs_tran', 'obs_test', 'real_val_on_grid', 'tran_time_pts', 'test_time_pts', 'time_grid'], 
                             verbose=False))
    datas.append(namedtuple('X3_data', ['obs_tran', 'obs_test', 'real_val_on_grid', 'tran_time_pts', 'test_time_pts', 'time_grid'], 
                             verbose=False))
    datas.append(namedtuple('Y_data', ['obs_tran', 'obs_test', 'real_val_on_grid', 'tran_time_pts', 'test_time_pts', 'time_grid'], 
                             verbose=False))

    for j in range(len(datas)):
        datas[j].obs_tran = np.loadtxt('tran '+ name[j] + ' of samp ' + np.str(i + 1),delimiter = ' ').T
        datas[j].obs_test = np.loadtxt('test '+ name[j] + ' of samp ' + np.str(i + 1), delimiter = ' ').T
        datas[j].real_val_on_grid = np.loadtxt('real_on_grid '+ name[j] + ' of samp ' + np.str(i + 1), delimiter = ' ').T
        datas[j].tran_time_pts = np.loadtxt('tran_time_pt '+ name[j] + ' of samp ' + np.str(i + 1), delimiter = ' ').T
        datas[j].test_time_pts = np.loadtxt('test_time_pt '+ name[j] + ' of samp ' + np.str(i + 1), delimiter = ' ').T
    X1_data.append(datas[0])
    X2_data.append(datas[1])
    X3_data.append(datas[2])
    Y_data.append(datas[3])
    
    X1_data[i].time_grid, X2_data[i].time_grid, X3_data[i].time_grid = time_grid_X1, time_grid_X2, time_grid_X3
    Y_data[i].time_grid = time_grid_Y

    # In[11]:

    '''
    Results of X1
    '''


    # In[13]:


    #------------------------
    # Get FPCA result of X1
    #------------------------

    result_X1 = FFReg.Get_FPCA_Result(X1_data[i], X1_candidate_h_mean, 
                                      X1_candidate_h_cov, X1_candidate_h_diag_cov, 
                                      ker_fun = "Gaussian",
                                      fve = 0.75)

    #fitting test sample
    fit_test_X1 =result_X1.Restruect_Fun(X1_data[i].test_time_pts.reshape(test_samp_size, num_pts, 1),
                                         X1_data[i].obs_test)[1]


    # In[17]:

    '''
    Results of X2
    '''


    # In[19]:

    #------------------------
    # Get FPCA result of X2
    #------------------------
    result_X2 = FFReg.Get_FPCA_Result(X2_data[i], X2_candidate_h_mean, 
                                      X2_candidate_h_cov, X2_candidate_h_diag_cov,
                                      ker_fun = "Gaussian",
                                      fve = 0.675)



    # In[21]:

    fit_test_X2 =result_X2.Restruect_Fun(X2_data[i].test_time_pts.reshape(test_samp_size, num_pts, 1),
                                         X2_data[i].obs_test)[1]


    # fit_train_X2 =result_X2.Restruect_Fun(X2_data[i].tran_time_pts.reshape(tran_samp_size, num_pts, 1),
    #                                      X2_data[i].obs_tran)[1]


    # In[22]:

    '''
    Results of X3
    '''


    # In[24]:

    #------------------------
    # Get FPCA result of X3
    #------------------------

    result_X3 = FFReg.Get_FPCA_Result(X3_data[i], 
                                      X3_candidate_h_mean, 
                                      X3_candidate_h_cov, 
                                      X3_candidate_h_diag_cov, 
                                      ker_fun = "Gaussian",
                                      fve = 0.7)

    # In[26]:

    fit_test_X3 =result_X3.Restruect_Fun(X3_data[i].test_time_pts.reshape(test_samp_size, num_pts, 1),
                                         X3_data[i].obs_test)[1]

    # fit_train_X3 =result_X3.Restruect_Fun(X3_data[i].tran_time_pts.reshape(tran_samp_size, num_pts, 1),
    #                                      X3_data[i].obs_tran)[1]

    '''
    Results of Y
    '''


    #------------------------
    # Get FPCA result of Y
    #------------------------
    result_Y = FFReg.Fit_Mean_and_Cov(Y_data[i], Y_candidate_h_mean, 
                                      Y_candidate_h_cov, ker_fun = "Gaussian")
    '''
    Construct Coefficient Function

        1. fit cov_Xi_Xj
        2. fit cov_Xi_Y
        3. combine them to build coefficient function

    '''


    # In[33]:

    fit_cov_X1_Y = FFReg.Fit_Cov_XY(X_time_pts = X1_data[i].tran_time_pts,
                                    Y_time_pts = Y_data[i].tran_time_pts,
                                    obs_X = X1_data[i].obs_tran, 
                                    obs_Y = Y_data[i].obs_tran,
                                    X_time_grid = X1_data[i].time_grid,
                                    Y_time_grid = Y_data[i].time_grid, 
                                    fit_X_mean = result_X1.mean_fun, 
                                    fit_Y_mean = result_Y.mean_fun, 
                                    candidate_h_cov = X1_Y_candidate_h_cov, 
                                    ker_fun = "Gaussian")


    # In[36]:

    # fit Cov
    fit_cov_X2_Y = FFReg.Fit_Cov_XY(X_time_pts = X2_data[i].tran_time_pts,
                                    Y_time_pts = Y_data[i].tran_time_pts,
                                    obs_X = X2_data[i].obs_tran, 
                                    obs_Y = Y_data[i].obs_tran,
                                    X_time_grid = X2_data[i].time_grid,
                                    Y_time_grid = Y_data[i].time_grid,
                                    fit_X_mean = result_X2.mean_fun,
                                    fit_Y_mean = result_Y.mean_fun,
                                    candidate_h_cov = X2_Y_candidate_h_cov, 
                                    ker_fun = "Gaussian")


    # In[39]:

    fit_cov_X3_Y = FFReg.Fit_Cov_XY(X_time_pts = X3_data[i].tran_time_pts,
                                    Y_time_pts = Y_data[i].tran_time_pts,
                                    obs_X = X3_data[i].obs_tran, 
                                    obs_Y = Y_data[i].obs_tran,
                                    X_time_grid = X3_data[i].time_grid,
                                    Y_time_grid = Y_data[i].time_grid,
                                    fit_X_mean = result_X3.mean_fun,
                                    fit_Y_mean = result_Y.mean_fun,
                                    candidate_h_cov = X3_Y_candidate_h_cov, 
                                    ker_fun = "Gaussian")


    # In[42]:

    fit_cov_X1_X2 = FFReg.Fit_Cov_XY(X_time_pts = X1_data[i].tran_time_pts,
                                     Y_time_pts = X2_data[i].tran_time_pts,
                                     obs_X = X1_data[i].obs_tran,
                                     obs_Y = X2_data[i].obs_tran,
                                     X_time_grid = X1_data[i].time_grid,
                                     Y_time_grid = X2_data[i].time_grid,
                                     fit_X_mean = result_X1.mean_fun,
                                     fit_Y_mean = result_X2.mean_fun,
                                     candidate_h_cov = X1_X2_candidate_h_cov, 
                                     ker_fun = "Gaussian")


    # In[45]:

    fit_cov_X1_X3 = FFReg.Fit_Cov_XY(X_time_pts = X1_data[i].tran_time_pts,
                                     Y_time_pts = X3_data[i].tran_time_pts,
                                     obs_X = X1_data[i].obs_tran,
                                     obs_Y = X3_data[i].obs_tran,
                                     X_time_grid = X1_data[i].time_grid,
                                     Y_time_grid = X3_data[i].time_grid,
                                     fit_X_mean = result_X1.mean_fun, 
                                     fit_Y_mean = result_X3.mean_fun, 
                                     candidate_h_cov = X1_X3_candidate_h_cov, 
                                     ker_fun = "Gaussian")


    # In[48]:

    fit_cov_X2_X3 = FFReg.Fit_Cov_XY(X_time_pts = X2_data[i].tran_time_pts,
                                     Y_time_pts = X3_data[i].tran_time_pts,
                                     obs_X = X2_data[i].obs_tran, 
                                     obs_Y = X3_data[i].obs_tran,
                                     X_time_grid = X2_data[i].time_grid,
                                     Y_time_grid = X3_data[i].time_grid, 
                                     fit_X_mean = result_X2.mean_fun, 
                                     fit_Y_mean = result_X3.mean_fun, 
                                     candidate_h_cov = X2_X3_candidate_h_cov, 
                                     ker_fun = "Gaussian")


    # In[50]:

    '''
    Coefficient Function
    '''


    # In[51]:

    #==========================================
    # Get Block_cov_XX (combine all cov_Xi_Xj)
    #==========================================


    block_cov_XX = np.bmat([[result_X1.cov_fun, fit_cov_X1_X2, fit_cov_X1_X3],
                            [fit_cov_X1_X2.T, result_X2.cov_fun, fit_cov_X2_X3],
                            [fit_cov_X1_X3.T, fit_cov_X2_X3.T, result_X3.cov_fun]])



    # In[54]:

    inv_block_cov_XX = np.linalg.pinv(block_cov_XX, rcond= 0.1)


    # In[56]:

    #==========================================
    # Get Block_cov_XY (combine all cov_Xi_Yj)
    #==========================================

    block_cov_XY = np.bmat([[fit_cov_X1_Y], [fit_cov_X2_Y], [fit_cov_X3_Y]])


    # In[59]:

    # -----------------------------
    # Compute Coefficient Function
    # -----------------------------

    fit_Beta_without_delta =  np.matmul(inv_block_cov_XX,block_cov_XY)

    ## add delta
    delta = np.diag(np.repeat([X1_data[i].time_grid[1] - X1_data[i].time_grid[0],
                               X2_data[i].time_grid[1] - X2_data[i].time_grid[0],
                               X3_data[i].time_grid[1] - X3_data[i].time_grid[0]],
                               [num_grid_X1, num_grid_X2, num_grid_X3]))

    delta_inv = np.linalg.inv(delta)
    fit_Beta  = np.matmul(delta_inv, fit_Beta_without_delta)

    '''
    1. Fit the Response on grid pts (for test Y)
    2. Plot the Result
    '''


    #---------------------------------------
    # fit Response of test set on grid
    #---------------------------------------
    fit_test_X_center = np.bmat([fit_test_X1 - result_X1.mean_fun,
                                 fit_test_X2 - result_X2.mean_fun,
                                 fit_test_X3 - result_X3.mean_fun])


    # In[67]:

    #compute test_Y(on grid pts)
    fit_test_Y = result_Y.mean_fun + np.matmul(fit_Beta_without_delta.T, fit_test_X_center.T).T



    # In[69]:

    #real value of test_Y (on grid pts)
    real_test_Y = Y_data[i].real_val_on_grid[tran_samp_size:,:]


    grid_width = (Y_data[i].time_grid[1] - Y_data[i].time_grid[0])
    #======================================
    #compute Mean integrated squared error
    #======================================

    my_MISE[i] = np.sum(np.square(real_test_Y - fit_test_Y) * grid_width, 1).mean()
    print(i, ', time = ', time.time() - start_time)
    np.savetxt('MISE_1y3x_sparse', my_MISE, newline='\n', delimiter = ' ', fmt="%s")


/pkg/cfd/anaconda3/lib/python3.6/site-packages/scipy/interpolate/interpolate.py:2469: RuntimeWarning: invalid value encountered in less
  out_of_bounds += x < grid[0]
/pkg/cfd/anaconda3/lib/python3.6/site-packages/scipy/interpolate/interpolate.py:2470: RuntimeWarning: invalid value encountered in greater
  out_of_bounds += x > grid[-1]
/pkg/cfd/anaconda3/lib/python3.6/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:161: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


Bandwidth of cov:  [ 0.4  0.3]
Bandwidth of cov:  [ 0.8  0.3]
Bandwidth of cov:  [ 0.7  0.4]
Bandwidth of cov:  [ 0.3  0.8]
Bandwidth of cov:  [ 0.4  0.9]
Bandwidth of cov:  [ 0.6  0.5]
0 , time =  58.25926899909973
Bandwidth of cov:  [ 0.4  0.4]
Bandwidth of cov:  [ 0.8  0.3]
Bandwidth of cov:  [ 0.7  0.3]
Bandwidth of cov:  [ 0.3  0.8]
Bandwidth of cov:  [ 0.4  0.9]
Bandwidth of cov:  [ 0.5  0.5]
1 , time =  56.4139130115509
Bandwidth of cov:  [ 0.4  0.4]
Bandwidth of cov:  [ 0.5  0.3]
Bandwidth of cov:  [ 0.7  0.4]
Bandwidth of cov:  [ 0.3  0.8]
Bandwidth of cov:  [ 0.5  0.5]
Bandwidth of cov:  [ 0.5  0.5]
2 , time =  56.35541558265686
Bandwidth of cov:  [ 0.4  0.4]
Bandwidth of cov:  [ 0.8  0.3]
Bandwidth of cov:  [ 0.7  0.4]
Bandwidth of cov:  [ 0.3  0.8]
Bandwidth of cov:  [ 0.3  0.9]
Bandwidth of cov:  [ 0.5  0.5]
3 , time =  56.451120138168335
Bandwidth of cov:  [ 0.3  0.4]
Bandwidth of cov:  [ 0.8  0.3]
Bandwidth of cov:  [ 0.7  0.4]
Bandwidth of cov:  [ 0.3  0.8]
Bandwidth of

In [2]:
my_MISE = np.loadtxt('MISE_1y3x_sparse', delimiter = ' ')

In [3]:
# create the quantile value of each array
# save in tuple

quantile_my_MISE  = []

#add mean        
quantile_my_MISE.append(np.asscalar(np.mean(my_MISE)))
#add median
quantile_my_MISE.append(np.asscalar(np.median(my_MISE)))

#add std
quantile_my_MISE.append(np.asscalar(np.std(my_MISE)))
#convert to tuple
quantile_my_MISE = tuple(quantile_my_MISE)

In [4]:
# print table
print("====================================================")
print("      Compare Mean Integrated Squared Error         ")
print("====================================================")
print("|   n = 200    |   mean   |   median   |    std    |")
print("----------------------------------------------------")
print("   my  method  " ,3 * " %10.3f"%quantile_my_MISE[:3])
print("----------------------------------------------------")

      Compare Mean Integrated Squared Error         
|   n = 200    |   mean   |   median   |    std    |
----------------------------------------------------
   my  method        24.164     23.878      4.028
----------------------------------------------------
